## Step 1: Install Dependencies

We only need `httpx` for making HTTP requests to the LLM.

In [1]:
!pip install httpx

## Step 2: Your First LLM Call

LM Studio provides an OpenAI-compatible API. That means:
- Endpoint: `http://localhost:1234/v1/chat/completions`
- Format: Same as OpenAI's API
- Local: Runs on your machine, no API keys needed!

Let's make a simple call:

In [2]:
import httpx
import json

# API configuration
BASE_URL = "http://localhost:1234/v1"
MODEL = "local-model"  # LM Studio uses this generic name

# Make a simple request
async def call_llm(prompt: str) -> str:
    """Call LM Studio with a simple prompt"""
    
    async with httpx.AsyncClient(timeout=60.0) as client:
        response = await client.post(
            f"{BASE_URL}/chat/completions",
            json={
                "model": MODEL,
                "messages": [
                    {"role": "user", "content": prompt}
                ],
                "temperature": 0.7,
                "max_tokens": 100,
            },
        )
        
        result = response.json()
        return result["choices"][0]["message"]["content"]

# Test it!
response = await call_llm("Say hello in a friendly way!")
print(response)

Hey there! 👋 Hope you're having a fantastic day!


## Step 3: Understand the Response

The LLM returns a JSON response. Let's look at the full structure:

In [3]:
async def call_llm_full(prompt: str) -> dict:
    """Call LLM and return full response"""
    
    async with httpx.AsyncClient(timeout=60.0) as client:
        response = await client.post(
            f"{BASE_URL}/chat/completions",
            json={
                "model": MODEL,
                "messages": [{"role": "user", "content": prompt}],
                "temperature": 0.7,
            },
        )
        return response.json()

# See the full structure
full_response = await call_llm_full("What is 2+2?")
print(json.dumps(full_response, indent=2))

{
  "id": "chatcmpl-ea2zckudyqj4otlkn0tlta",
  "object": "chat.completion",
  "created": 1769779895,
  "model": "openai/gpt-oss-20b",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The answer is **4**.",
        "reasoning": "Simple math: 4.",
        "tool_calls": []
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 74,
    "completion_tokens": 23,
    "total_tokens": 97
  },
  "stats": {},
  "system_fingerprint": "openai/gpt-oss-20b"
}


## Step 4: Real Use Case - Classify a Tender

Now let's use the LLM for something useful: determining if a procurement tender is relevant to our tech company.

In [4]:
async def classify_tender(title: str, description: str) -> str:
    """Use LLM to classify if a tender is relevant"""
    
    prompt = f"""
You are a procurement analyst. Is this tender relevant for a tech company
specializing in AI, cybersecurity, and software development?

TENDER TITLE: {title}

DESCRIPTION: {description}

Answer with "RELEVANT" or "NOT RELEVANT" and explain why in one sentence.
"""
    
    return await call_llm(prompt)

# Test with real examples
print("Example 1: AI Cybersecurity Tender")
print("=" * 50)
result1 = await classify_tender(
    title="AI-Powered Threat Detection System",
    description="Government needs AI system to detect cybersecurity threats in real-time."
)
print(result1)

print("\n\nExample 2: Office Furniture")
print("=" * 50)
result2 = await classify_tender(
    title="Office Furniture Supply",
    description="Supply 500 ergonomic chairs and desks for government building."
)
print(result2)

Example 1: AI Cybersecurity Tender
RELEVANT – The tender seeks an AI‑driven system for real‑time threat detection, directly aligning with the company’s expertise in AI, cybersecurity, and software development.


Example 2: Office Furniture
NOT RELEVANT – the tender is for ergonomic office furniture, which does not involve AI, cybersecurity, or software development services that the company specializes in.


## 🎉 Congratulations!

You just:
1. Made your first LLM API call
2. Understood the response format
3. Built a real tender classifier

## Key Takeaways

- **LLMs are just HTTP APIs** - You send text, get text back
- **Temperature controls randomness** - Lower = more consistent
- **Prompts are instructions** - Clear prompts = better results

## Next Steps

In the next notebook, we'll learn about **structured outputs** using Pydantic to get JSON instead of free text. This is crucial for building reliable AI systems!

➡️ Continue to `02_structured_outputs.ipynb`